<a href="https://www.kaggle.com/code/sahidvelji/self-follows-on-kaggle?scriptVersionId=98953495" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Self-follows on Kaggle
The follow feature on Kaggle was created by a Google engineer as a short [20% project](https://en.wikipedia.org/wiki/20%25_Project) as described [here](https://kaggle.com/general/33512).
A while ago I noticed that some users managed to follow themselves. I decided to take a closer look using the [Meta Kaggle](https://kaggle.com/kaggle/meta-kaggle) dataset.

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
from IPython.display import display, HTML
from pathlib import Path

px.defaults.template = 'plotly_white'
px.defaults.color_discrete_sequence = ['steelblue']
MODE_BAR_BUTTONS = ['toImage', 'zoom2d', 'pan2d', 'select2d', 'lasso2d',
                    'zoomIn2d', 'zoomOut2d', 'autoScale2d', 'resetScale2d',
                    'toggleSpikelines', 'hoverClosestCartesian', 'hoverCompareCartesian'
                   ]
CONFIG = {
    'modeBarButtonsToRemove': ['pan2d', 'select2d', 'lasso2d', 'toggleSpikelines']
}

KAGGLE_BASE_URL = 'https://www.kaggle.com/'

PERFORMANCE_TIERS = {0: 'Novice',
                     1: 'Contributor',
                     2: 'Expert',
                     3: 'Master',
                     4: 'Grandmaster',
                     5: 'Staff'
                    }
avatiers = {i: f'<img src="https://kaggle.com/static/images/avatier/avatier-{PERFORMANCE_TIERS[i].lower()}@2x.png">' for i in range(6)}

In [3]:
path = Path('/kaggle/input/meta-kaggle')

users = pd.read_csv(path/'Users.csv',
                    parse_dates=['RegisterDate'], 
                    infer_datetime_format=True
                   )
user_followers = pd.read_csv(path/'UserFollowers.csv', 
                             parse_dates=['CreationDate'], 
                             infer_datetime_format=True
                            )

In [4]:
self_follows = (user_followers[user_followers.UserId.eq(user_followers.FollowingUserId)]
                .merge(users, left_on='UserId', right_on='Id')
                .drop(columns=['Id_x', 'Id_y', 'UserId', 'FollowingUserId'])
                .rename(columns={'CreationDate': 'FollowDate'})
                .sort_values('FollowDate', ignore_index=True)
                .assign(PerfTier=lambda x: x['PerformanceTier'].replace(avatiers),
                        User=lambda x: '<a href="' + KAGGLE_BASE_URL + x.UserName + '">' + x.DisplayName + '</a>' + x.PerfTier)
                .replace({'PerformanceTier': PERFORMANCE_TIERS})
                .reindex(columns=['FollowDate', 'User', 'RegisterDate', 'PerformanceTier'])
               )
self_follows.index = self_follows.index + 1

self_follow_string = f'There are currently {self_follows.shape[0]} users that follow themselves.'
display(HTML(self_follow_string))
display(HTML(self_follows.to_html(columns=['FollowDate', 'User', 'RegisterDate'],
                                  escape=False,
                                  render_links=True
                                 ).replace('<td>', '<td style="text-align:left">')))

,FollowDate,User,RegisterDate
1,2017-12-30,John Gonzalez,2017-12-30
2,2018-12-29,Erik Bruin,2017-11-24
3,2019-01-26,John Carlo,2018-10-14
4,2019-02-07,Ironmatt,2018-12-30
5,2019-02-11,Vincent Lugat,2017-03-06
6,2019-02-15,Kshitijgarg,2018-12-15
7,2019-02-20,Makragic,2018-10-01
8,2019-02-22,Roman Slaap,2017-05-18
9,2019-02-23,EnricRovira,2017-03-01
10,2019-02-24,Muhammed Muqtar Fasasi,2019-02-24


In [5]:
fig = px.scatter(self_follows,
                 x='FollowDate',
                 y=range(1, self_follows.shape[0] + 1)
                )
fig.update_traces(mode='lines',
                  hovertemplate=
                  '<b>%{x}</b><br>'+
                  'Number of self-follows: <b>%{y}</b>'
                 )
fig.update_layout(title='Number of self-follows over time',
                  xaxis_title='Date',
                  yaxis_title='Number of self-follows',
                  hoverlabel_bgcolor='white',
                  hoverlabel_font_size=14,
                  hovermode="x",
                  yaxis_zerolinecolor='grey',
                  yaxis_zerolinewidth=1
                 )
fig.show(config=CONFIG)

It looks like the first self-follow occurred on 30 December 2017. The second self-follow happened almost a year later. After the second self-follow, many other users begin following themselves.

In [6]:
self_follows_by_tier = (self_follows
                        .PerformanceTier
                        .value_counts()
                        .to_frame()
                        .reset_index()
                        .rename(columns={'index': 'PerformanceTier', 'PerformanceTier': 'Count'})
                       )
fig = px.bar(self_follows_by_tier,
             x='Count',
             y='PerformanceTier',
             orientation='h'
            )
fig.update_traces(hovertemplate=
                  'Performance tier: <b>%{y}</b><br>'+
                  'Count: <b>%{x}</b><br>',
                  texttemplate='%{x}',
                  textposition='outside',
                  cliponaxis=False
                 )
fig.update_layout(title='Number of self-follows by performance tier',
                  xaxis_title='Number of self-follows',
                  yaxis_title='Performance tier',
                  yaxis_autorange='reversed',
                  hoverlabel_bgcolor="white",
                  hoverlabel_font_size=14,
                  hovermode="y"                  
                 )
fig.show(config=CONFIG)

I have not tried following myself but I found this [thead](https://kaggle.com/getting-started/126849) where a user describes being able to follow himself (but unable to revert this action) after finding his own notebook on the newsfeed. Since there is a "Follow" button next to users on the newsfeed, you could follow yourself if you happen to find your own work there.

Update (2020-08-31): With the [Profile Preview](https://www.kaggle.com/product-feedback/177372) update launched on Tuesday 25 August 2020, it is now much easier to follow yourself. Simply hover over your user avatar and click on "Follow" when the profile preview pops up.